In [83]:
import pandas as pd
# Load data from the "Usage" sheet
file_path = 'MO14-Round-1-Dealing-With-Data-Workbook.xlsx'

usage_data = pd.read_excel(file_path, sheet_name='Usage')
contracts_df = pd.read_excel(file_path, sheet_name='Contracts')

# Display the first few rows of the usage data
usage_data.head()

D:\Users\dell\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,3 PM Mon 24th-Mar-2014___0.384 kwh
0,5AM 15-Aug-2014___1.201 kwh
1,__8PM Thu 20-Mar-2014____1.523 kwh
2,6PM 23rd-Apr-2014___0.424 kwh
3,_1AM Friday 19th-Dec-2014___0.209 kwh
4,_5AM Tue 19th-Aug-2014___1.228 kwh


In [92]:
contracts_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
# Iterate through all sheets and print the first few rows to find the contract details
for sheet_name in sheet_names:
    print(f"Inspecting sheet: {sheet_name}")
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    print(df.head(10))
    print("\n")


Inspecting sheet: Contracts
  Unnamed: 0  Unnamed: 1 Unnamed: 2  Unnamed: 3    Unnamed: 4  Unnamed: 5  \
0        NaN         NaN        NaN         NaN           NaN         NaN   
1        NaN         NaN        NaN         NaN           NaN         NaN   
2        NaN         NaN        NaN         NaN           NaN         NaN   
3        NaN         NaN        NaN         NaN           NaN         NaN   
4        NaN         NaN        NaN         NaN           NaN         NaN   
5        NaN         NaN        NaN         NaN           NaN         NaN   
6        NaN         NaN        NaN         NaN           NaN         NaN   
7        NaN         NaN        NaN         NaN           NaN         NaN   
8                    NaN        NaN         NaN           NaN         NaN   
9    No Flex         NaN        NaN         NaN  Monthly Flex         NaN   

  Unnamed: 6  Unnamed: 7   Unnamed: 8  Unnamed: 9 Unnamed: 10  
0        NaN         NaN          NaN         NaN         Na

D:\Users\dell\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


       3 PM  Mon 24th-Mar-2014___0.384 kwh  
0             5AM  15-Aug-2014___1.201  kwh 
1       __8PM Thu 20-Mar-2014____1.523 kwh  
2            6PM  23rd-Apr-2014___0.424  kwh
3  _1AM  Friday 19th-Dec-2014___0.209  kwh  
4      _5AM  Tue 19th-Aug-2014___1.228  kwh 
5        12PM  Mon 7th-Jul-2014___1.296 kwh 
6           7 AM  Tue 25-Nov-2014__0.437 kwh
7              _8AM  14-Aug-2014__0.523 kwh 
8           __4PM  25th-Jan-2014__2.052kwh  
9             4PM   11th-Feb-2014_0.509 kwh 




D:\Users\dell\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [110]:
import re

# Function to clean and extract data from each row with better handling for shifted values and preserving day of the week
def final_clean_usage_data(row):
    row = row.strip()  # Remove leading and trailing whitespaces
    
    # Extract usage (last number in the row)
    usage_match = re.search(r'(\d+\.\d+)\s*kwh', row, re.IGNORECASE)
    if usage_match:
        usage = float(usage_match.group(1))
    else:
        usage = None
    
    # Remove the usage part to focus on hour, date, and day
    row = row.replace(usage_match.group(), '').strip() if usage_match else row
    
    # Extract hour (if it appears at the start or within the string)
    hour_match = re.search(r'(_* *\d{1,2}\s*[APM]{2})', row, re.IGNORECASE)
    if hour_match:
        hour = hour_match.group(1).strip()
        date_str = row.replace(hour, '').strip()
    else:
        hour = None
        date_str = row.strip()
    
    # Extract date (matches common date formats) and day
    date_match = re.search(r'([A-Za-z]{3,9}\s)?(\d{1,2}[a-z]{2}-[A-Za-z]{3}-\d{4}|\d{1,2}-[A-Za-z]{3}-\d{4}|\d{1,2}\s[A-Za-z]{3}\s\d{4})', date_str)
    if date_match:
        date = date_match.group(2)
        # Extract day of the week if available
        day_match = re.search(r'([A-Za-z]{3,9})', date_match.group(1)) if date_match.group(1) else None
        day = day_match.group(1) if day_match else None
    else:
        date = None
        day = None
    
    return hour, date, usage, day

# Apply the final cleaning function to each row
final_cleaned_data = usage_data.iloc[:, 0].apply(final_clean_usage_data)

# Convert cleaned data to a DataFrame
final_cleaned_usage_df = pd.DataFrame(final_cleaned_data.tolist(), columns=['Hour', 'Date', 'Usage', 'Day'])

# Display the first few rows of the cleaned data
final_cleaned_usage_df.head(10)


,Hour,Date,Usage,Day
0,5AM,15-Aug-2014,1.201,None
1,__8PM,20-Mar-2014,1.523,Thu
2,6PM,23rd-Apr-2014,0.424,None
3,_1AM,19th-Dec-2014,0.209,Friday
4,_5AM,19th-Aug-2014,1.228,Tue
5,12PM,7th-Jul-2014,1.296,Mon
6,7 AM,25-Nov-2014,0.437,Tue
7,_8AM,14-Aug-2014,0.523,None
8,__4PM,25th-Jan-2014,2.052,None
9,4PM,11th-Feb-2014,0.509,None


In [56]:
final_cleaned_usage_df.isna().sum()

Hour        0
Date        0
Usage       0
Day      4301
dtype: int64

In [85]:
final_cleaned_usage_df['Day'][8758] = 'Thursday'
final_cleaned_usage_df['Day'].fillna(method='bfill', inplace=True)

final_cleaned_usage_df.head()

C:\Users\dell\AppData\Local\Temp/ipykernel_13268/66946956.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_cleaned_usage_df['Day'][8758] = 'Thursday'


,Hour,Date,Usage,Day
0,5AM,15-Aug-2014,1.201,Thu
1,__8PM,20-Mar-2014,1.523,Thu
2,6PM,23rd-Apr-2014,0.424,Friday
3,_1AM,19th-Dec-2014,0.209,Friday
4,_5AM,19th-Aug-2014,1.228,Tue


In [58]:
final_cleaned_usage_df.isna().sum()

Hour     0
Date     0
Usage    0
Day      0
dtype: int64

#### Question 1: What is your average hourly electricity usage?

In [100]:
# Calculate average hourly usage
average_hourly_usage = final_cleaned_usage_df['Usage'].mean()

# Print the result with appropriate formatting
print("Average hourly electricity usage:", average_hourly_usage, "kWh")


Average hourly electricity usage: 0.781923393081394 kWh


#### Question 2: What is your average electricity usage per hour in February?

In [88]:
february_data = final_cleaned_usage_df[final_cleaned_usage_df['Date'].dt.month == 2]

# Calculate the average usage per hour in February
average_february_usage = february_data['Usage'].mean()

print("The average electricity usage per hour in February is:", average_february_usage, "kWh")

The average electricity usage per hour in February is: 0.8329151785714305 kWh


#### Question 3: Which day of the week has the highest average usage? 

In [86]:
average_usage_per_day = final_cleaned_usage_df.groupby('Day')['Usage'].mean()
average_usage_per_day.idxmax()

'Saturday'

#### Question 4: What is the highest amount of electricity used in a continuous 4 hour period? 

In [111]:
def remove_ordinal_suffix(date_str):
    return date_str.replace('st', '').replace('nd', '').replace('rd', '').replace('th', '')

# Apply the function to the Date column
final_cleaned_usage_df['Date'] = final_cleaned_usage_df['Date'].apply(remove_ordinal_suffix)

# Parse the dates
final_cleaned_usage_df['Date'] = pd.to_datetime(final_cleaned_usage_df['Date'], format='%d-%b-%Y')

# Drop rows with invalid dates
final_cleaned_usage_df.dropna(subset=['Date'], inplace=True)

# Sort the data by Date and Hour to ensure the time sequence is correct
final_cleaned_usage_df = final_cleaned_usage_df.sort_values(by=['Date', 'Hour'])

# Reset the index after sorting
final_cleaned_usage_df.reset_index(drop=True, inplace=True)

# Calculate the sum of usage for every continuous 4-hour period
highest_4_hour_usage = final_cleaned_usage_df['Usage'].rolling(window=4).sum().max()

print("The highest amount of electricity used in a continuous 4-hour period is:", highest_4_hour_usage, "kWh")


The highest amount of electricity used in a continuous 4-hour period is: 15.782000000000004 kWh


#### Question 6: Based on your historic electricity usage, which of the three contracts would produce the lowest annual cost?

 By Looking at the sheet and analyzing their average The Hourly Flex plan could produce the lowest annual cost